In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import holoviews as hv
import geoviews as gv
import iris
import cartopy

from cartopy import crs
from cartopy import feature as cf
from geoviews import feature as gf

hv.notebook_extension('bokeh','matplotlib')
# %output backend='matplotlib'
# %opts Feature [projection=crs.Robinson()]



In [11]:
iris.FUTURE.netcdf_promote=True
iris_ensemble = iris.load_cube('/d4/yzhou/Drought_atlas/MADA/mada_25.nc')
print iris_ensemble.summary()

pdsi / (unknown)                    (longitude: 34; latitude: 27; time: 706)
     Dimension coordinates:
          longitude                           x             -         -
          latitude                            -             x         -
          time                                -             -         x
     Attributes:
          invalid_units: unitless


In [87]:
iris_dataset = gv.Dataset(iris_ensemble)
print repr(iris_dataset)

:Dataset   [T,longitude,latitude]   (pdsi)


In [99]:
gv_image = iris_dataset.to(gv.Image, ['longitude', 'latitude'])

In [117]:
# %opts Image [colorbar=True]
%output size=150
gv_image[521:523]

:HoloMap   [T]
   :Image   [longitude,latitude]   (pdsi)

In [13]:
iris_ensemble.transpose() # Transpose the data so that time is the first dimension. Otherwise eofs doesn't work

In [14]:
from eofs.iris import Eof
# create a solver class
solver = Eof(iris_ensemble)

In [15]:
# retrieve the first few EOFs from the solver class
eofs = solver.eofs(neofs=5)

In [16]:
eofs_dataset = gv.Dataset(eofs)

In [17]:
# %opts Image [colorbar=True]
%output size=150
eofs_to = eofs_dataset.to(gv.Image, ['longitude', 'latitude']) * gf.coastline()
eofs_to.layout()

:NdLayout   [eof_number]
   :Overlay
      .Empirical_orthogonal_functions.I :Image   [longitude,latitude]   (empirical_orthogonal_functions)
      .Coastline.I                      :Feature   [Longitude,Latitude]

In [18]:
pcs = solver.pcs(npcs=5, pcscaling=1)
pcs.transpose() # so hv.Curve plots correctly

In [19]:
hv.Curve(pcs[0]) + hv.Curve(pcs[1]) + hv.Curve(pcs[2]) + hv.Curve(pcs[3]) + hv.Curve(pcs[4])

:Layout
   .Principal_components.I   :Curve   [time]   (principal_components)
   .Principal_components.II  :Curve   [time]   (principal_components)
   .Principal_components.III :Curve   [time]   (principal_components)
   .Principal_components.IV  :Curve   [time]   (principal_components)
   .Principal_components.V   :Curve   [time]   (principal_components)